In [1]:
using EzXML
using Plots
using StatsPlots
using NPZ
using Statistics

# Enveloping

An envelope of time series data is a plot which passes through all the local maxima/minima in the time series data. This is a better representation of the LFP data in relation to utility in a relational model between LFP and O2 data. This notebook will import a set of the LPF data, separate it into neural brain frequency bands, and then compute the envelopes of each resulting time series data.

## LFP Data Importing

In [2]:
# Get the XML metadata file

lfp_path = "Exp 2/R7/2022-05-11_17-39-17/Record Node 101/"
files_xml = root(readxml("../../data/$(lfp_path)Continuous_Data.openephys"))
sample_rate = parse(Int, firstelement(files_xml)["samplerate"])

recording = firstelement(firstelement(files_xml))

# Format XML data into Dict
files = Dict("processor" => recording["id"], "name" => [], "filename" => [], "position" => [], "bitVolts" => [])
for elem in eachelement(recording)
    push!(files["name"], elem["name"])
    filename = replace(elem["filename"], r"continuous$"=>"")
    push!(files["filename"], "$(filename)npz")
    push!(files["position"], elem["position"])
    push!(files["bitVolts"], elem["bitVolts"])
end

In [3]:
# Indexing lfp_data[channel, datapoint]

# Add data from each channel into a single matrix
n_datapoints = length(npzread("../../data/formatted-lfp/$(lfp_path)$(files["filename"][1])", ["data"])["data"][1:end-1024])
n_channels = length(files["filename"])
lfp_data = Array{Float32}(undef, n_channels, n_datapoints)
for i in 1:n_channels
    data = npzread("../../data/formatted-lfp/$(lfp_path)$(files["filename"][i])", ["data"])["data"][1:end-1024] .* parse(Float64, files["bitVolts"][i])
    lfp_data[i, :] = data
end

# Normalisation
for i in 1:n_channels
    lfp_data[i, :] = (lfp_data[i, :] .- mean(lfp_data[i, :])) ./ std(lfp_data[i, :])
end

In [4]:
# Add timestamps into vector 
# Timestamps are identical for each channel so this only needs to be done once
lfp_timestamps = npzread("../../data/formatted-lfp/$(lfp_path)$(files["filename"][1])", ["timestamps"])["timestamps"]
n_timestamps = length(lfp_timestamps)

lfp_timestamps_extended = [lfp_timestamps[i-1] + (lfp_timestamps[i] - lfp_timestamps[i-1]) * j / 1024 for i in 2:n_timestamps for j in 1:1024]
lfp_timestamps_extended ./= sample_rate

@assert length(lfp_timestamps_extended) == n_datapoints

In [5]:
events_data = npzread("../../data/formatted-lfp/$(lfp_path)all_channels.npz")

Dict{String, Vector{Float64}} with 7 entries:
  "eventId"         => [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0  …  0.…
  "channel"         => [7.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0  …  4.…
  "nodeId"          => [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0,…
  "eventType"       => [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0  …  3.…
  "timestamps"      => [25998.0, 27999.0, 28499.0, 38504.0, 39004.0, 49009.0, 4…
  "sampleNumber"    => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.…
  "recordingNumber" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.…

In [6]:
flag_timestamps = events_data["timestamps"][findall(x->x==4, events_data["channel"])]./1000 
flag_eventIds = events_data["eventId"][findall(x->x==4, events_data["channel"])]

laser_timestamps = events_data["timestamps"][findall(x->x==0, events_data["channel"])]./1000 
laser_eventIds = events_data["eventId"][findall(x->x==0, events_data["channel"])]

lfp_flags = flag_timestamps[findall(x->x==1, flag_eventIds)]
lfp_laser = laser_timestamps[findall(x->x==1, laser_eventIds)]

17-element Vector{Float64}:
  326.7
  449.063
  494.585
  534.267
  626.499
  731.54
  817.746
 1050.248
 1137.999
 1216.344
 1299.674
 1382.94
 1466.053
 1524.079
 1623.609
 1623.611
 1623.613

## Neural Brain Frequency Separation

This section will separate out the raw LFP signal into the neural brain frequencies which are defined as follows:

$$\alpha=8-13Hz$$
$$\beta=18-25Hz$$
$$\theta=3.5-7Hz$$
$$\delta=0.5-3.5Hz$$
$$\gamma=30-70Hz$$

These definitions are taken from [Başar, 2013](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3811101/#:~:text=Natural%20brain%20frequencies%20are%20denoted,gamma%3A%2030%2D70%20Hz.&text=When%20the%20stimulus%20signal%20contains,oscillations%20are%20considered%20as%20ERO.).